In [22]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [23]:
#dataset
fmnist = keras.datasets.fashion_mnist
#preparing the training set and test set
(training_images, training_labels), (test_images, test_labels) = fmnist.load_data()

In [24]:
#shape of training set and test set
print("training_images shape : ", np.shape(training_images))
print("training_labels shape : ", np.shape(training_labels))
print("test_images shape : ", np.shape(test_images))
print("test_labels shape : ", np.shape(test_labels))

training_images shape :  (60000, 28, 28)
training_labels shape :  (60000,)
test_images shape :  (10000, 28, 28)
test_labels shape :  (10000,)


In [25]:
#input to the first conv layer - a greyscale (28,28,1) image
training_images = np.reshape(training_images, (60000, 28, 28, 1))
test_images = np.reshape(test_images, (10000, 28, 28, 1))

In [26]:
#normalizing the data
training_images = training_images/255
test_images = test_images/255

In [27]:
#Model : conv->pool->conv->pool->flatten->FC(ReLU)->FC(Softmax)
model = keras.models.Sequential([
                                 keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
                                 keras.layers.MaxPool2D(pool_size=(2,2), strides=2),
                                 keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                                 keras.layers.MaxPool2D(pool_size=(2,2), strides=2),
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(128, activation='relu'),
                                 keras.layers.Dense(10, activation='softmax')
])

In [28]:
#optimizer : Adam, loss-function : cross entropy loss
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               204928    
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [31]:
#code that stops training once the model reaches desired performance
class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      print("\nReached desired performance, so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [32]:
#training the network
model.fit(training_images, training_labels, epochs=5, callbacks=[callbacks])

Epoch 1/5
1875/1875 [==============================] - 81s 43ms/step - loss: 0.5954 - accuracy: 0.7801
Epoch 2/5
1875/1875 [==============================] - 79s 42ms/step - loss: 0.3067 - accuracy: 0.8891
Epoch 3/5
1875/1875 [==============================] - 79s 42ms/step - loss: 0.2458 - accuracy: 0.9079
Epoch 4/5
1875/1875 [==============================] - 79s 42ms/step - loss: 0.2170 - accuracy: 0.9212
Epoch 5/5
1875/1875 [==============================] - 79s 42ms/step - loss: 0.1866 - accuracy: 0.9307


In [33]:
#performance on the test set
model.evaluate(test_images, test_labels)

313/313 [==============================] - 4s 13ms/step - loss: 0.2626 - accuracy: 0.9094


[0.2626011073589325, 0.9093999862670898]